In [3]:
# Initialize Otter
import otter
grader = otter.Notebook("lab.ipynb")

In [2]:
import sys
del sys.path[4]
sys.modules['IPython']
del sys.modules['IPython']
del sys.modules['IPython.core']

# Lab 2 – DataFrames and Grouping

## DSC 80, Fall 2025

### Due Date: Monday, October 13th at 11:59PM

## Instructions

Welcome to the second DSC 80 lab this quarter!

Much like in DSC 10, this Jupyter Notebook contains the statements of the problems and provides code and Markdown cells to display your answers to the problems. Unlike DSC 10, the notebook is *only* for displaying a readable version of your final answers. The coding will be done in an accompanying `lab.py` file that is imported into the current notebook, and **you will only submit that `lab.py` file**, not this notebook!

Some additional guidelines:
- **Unlike in DSC 10, labs will have both public tests and hidden tests.** The bulk of your grade will come from your scores on hidden tests, which you will only see on Gradescope after the assignment deadline.
- **Do not change the function names in the `lab.py` file!** The functions in the `lab.py` file are how your assignment is graded, and they are graded by their name. If you changed something you weren't supposed to, you can find the original code in the [course GitHub repository](https://github.com/dsc-courses/dsc80-2025-fa).
- Notebooks are nice for testing and experimenting with different implementations before designing your function in your `lab.py` file. You can write code here, but make sure that all of your real work is in the `lab.py` file, since that's all you're submitting.
- You are encouraged to write your own additional helper functions to solve the lab, as long as they also end up in `lab.py`.

**To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.** More details on its usage are given at the bottom of this notebook.

**Importing code from `lab.py`**:

* Below, we import the `.py` file that's contained in the same directory as this notebook.
* We use the `autoreload` notebook extension to make changes to our `lab.py` file immediately available in our notebook. Without this extension, we would need to restart the notebook kernel to see any changes to `lab.py` in the notebook.
    - `autoreload` is necessary because, upon import, `lab.py` is compiled to bytecode (in the directory `__pycache__`). Subsequent imports of `lab` merely import the existing compiled python.

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from lab import *

In [6]:
import os
import io
from pathlib import Path
import pandas as pd
import numpy as np

<div class="alert alert-block alert-danger">

<b>The only question in this lab that you're allowed to use a loop in is Question 3.</b> There, you may use a <code>for</code>-loop to loop over the columns in the input DataFrame, but not the rows. <b>If you use a <code>for</code>-loop or <code>while</code>-loop in any other question, you may lose points!</b>

</div>

## Part 1: Tricky Pandas 🤔

Sometimes, `pandas` gives you weird outputs that you may not expect. The next question walks you through a few examples that might surprise you. 

### Question 1

The following subparts all require you to define a function and return a number that is the answer to a multiple-choice question. You may need to write code and experiment with DataFrames to arrive at your answers.

#### `trick_me`

`trick_me` should not take any arguments. 
<br>

Inside the function:

* Create a DataFrame named `tricky_1` that has three columns labeled `'Name'`, `'Name'`, and `'Age'`. `tricky_1` should have 5 rows; the values are up to you.
* Save the DataFrame to a `.csv` file called `'tricky_1.csv'` without the index.
* Now create another DataFrame, named `tricky_2`, by reading in the file `'tricky_1.csv'`. What are your observations?

  1. It was not possible to create a DataFrame with the duplicate columns.
  2. `tricky_1` and `tricky_2` have the same column names.
  3. `tricky_1` and `tricky_2` have different column names.
   
Your function should return `1`, `2`, or `3`, answering the above question.

<font color='red'>**Hints:** Dictionaries can only contain unique keys, so you <b>cannot</b> use them to declare a DataFrame with duplicate column names. Find other ways to create a DataFrame with duplicate column names.</font>

<br>
  
#### `trick_bool`
`trick_bool` should not take any arguments.

To determine the correct answer from the list below, you should follow the steps outlined by experimenting in **the notebook** (or in the Terminal by running `python`). Outside the function:

* Create a DataFrame named `bools` that has four columns: `True`, `True`, `False`, `False`. Each column name should be Boolean.
* `bools` should have 4 rows; the values are up to you.
* Predict the shape of the DataFrame that results by running each of the three lines of code below. Pick a corresponding answer from the given list. Your function should return a list with three numbers, one for each line.
* You should be able to answer without running any code, but feel free to run code to check your answer.
* **Your function should not do anything other than return a hardcoded list.**

```py
bools[True]
bools[[True, True, False, False]]
bools[[True, False]]
```
    
Answer choices:
1. DataFrame: 2 columns, 1 row
2. DataFrame: 2 columns, 2 rows
3. DataFrame: 2 columns, 3 rows
4. DataFrame: 2 columns, 4 rows
5. DataFrame: 3 columns, 1 rows
6. DataFrame: 3 columns, 2 rows
7. DataFrame: 3 columns, 3 rows
8. DataFrame: 3 columns, 4 rows
9. DataFrame: 4 columns, 1 rows
10. DataFrame: 4 columns, 2 rows
11. DataFrame: 4 columns, 3 rows
12. DataFrame: 4 columns, 4 rows
13. Error

**Hints:** Refer to the previous hint for `trick_me`.

In [7]:
def trick_bool():
    return [4, 4, 12]


In [8]:
def trick_me():
    return 3

def trick_bool():
    return [4, 4, 12]


In [9]:
# don't change this cell -- it is needed for the tests to work
trick_ans = trick_bool()

In [10]:
grader.check("q1")

q1 results: All test cases passed!

## Part 2: Summary Statistics 📊

In this question you will define two general purpose functions that make it easy to qualitatively assess the contents of a DataFrame.

### Question 2

Complete the implementation of the function `population_stats`, which takes in a DataFrame `df` and returns a DataFrame indexed by the columns of `df`, with the following columns:
* `'num_nonnull'`, which contains the number of non-null entries in each column.
* `'prop_nonnull'`, which contains the proportion of entries in each column that are non-null.
* `'num_distinct'`, which contains the number of distinct non-null entries in each column.
* `'prop_distinct'`, which contains the proportion of non-null entries that are distinct in each column.
       
For example, if `df` has a column named `'ages'` with the following elements (note that `np.nan` is a null value):
       
```py
[2, 2, 2, np.nan, 5, 7, 5, 10, 11, np.nan]
```

Then:
- `'num_nonnull'` is 8, and `'prop_nonnull'` is $\frac{8}{10}$ = 0.8.
- There are six distinct entries, `[2, 5, 7, 10, 11, np.nan]`, but only 5 of them are non-null. So the number of distinct non-null entries, `'num_distinct'`, is 5.
- There are 5 distinct non-null entries, and there are 8 total non-null entries, so `'prop_distinct'` is $\frac{5}{8}$ = 0.625.

Putting it all together, `population_stats(df).loc['ages']` should be a Series containing the numbers 8, 0.8, 5, and 0.625.

In [11]:
def population_stats(df):
    # Gesamtzahl der Zeilen (für Proportionen)
    total = len(df)

    # Anzahl nicht-null Werte pro Spalte
    num_nonnull = df.notna().sum()

    # Anteil nicht-null Werte
    prop_nonnull = num_nonnull / total

    # Anzahl verschiedener nicht-null Werte
    num_distinct = df.nunique(dropna=True)

    # Anteil verschiedener Werte unter den nicht-null Werten
    prop_distinct = num_distinct / num_nonnull

    # Alles in ein gemeinsames DataFrame zusammenführen
    result = pd.DataFrame({
        'num_nonnull': num_nonnull,
        'prop_nonnull': prop_nonnull,
        'num_distinct': num_distinct,
        'prop_distinct': prop_distinct
    })

    return result


In [12]:
# don't change this cell -- it is needed for the tests to work
pop_data = np.random.choice(range(10), size=(100, 4))
df_pop = pd.DataFrame(pop_data, columns='A B C D'.split())
out_pop = population_stats(df_pop)

In [13]:
grader.check("q2")

q2 results: All test cases passed!

### Question 3
    
Complete the implementation of the function `most_common`, which takes in a DataFrame `df` and a number `N` and returns a DataFrame of the `N` most-common values and their counts for each column of `df`. Any column with fewer than `N` distinct values should contain `np.nan` in those entries.

For example, consider the DataFrame shown on the left. This DataFrame is a subset of `salaries`, a larger DataFrame containing information on employees in the City of San Diego. The subset below contains two of the original columns: `'Job Title'` which contains job titles for employees, and `'status'` which denotes whether the employee works a full time position (`'FT'`) or a part time position (`'PT'`). On the right, the return value of `most_common(salaries, N=5)` is shown.

You can assume that there are no ties in our hidden tests.

<table><tr>
    <td><img src="data/imgs/dataframe.png" width="90%"/></td>
    <td><img src="data/imgs/most_common.png" width="90%"/></td>
</tr></table>

***Note***: Remember, to access values in a Series based on their integer position, including when slicing for the first `N` values in a Series, you **must** use `.iloc` followed by square brackets. If you just use square brackets and don't use `.iloc`, you may not see the results you expect!

***Hint***: You may find that initializing an empty DataFrame with `N` rows and adding columns to it is useful in your implementation.

<div class="alert alert-block alert-danger">

<b>Remember, the only question in this lab that you're allowed to use a loop in is Question 3 – that's this question.</b> Here, you may use a <code>for</code>-loop to loop over the columns in the input DataFrame (<code>df</code>), but not the rows. <b>If you use a <code>for</code>-loop or <code>while</code>-loop in any other question, you may lose points!</b>

</div>

In [17]:
def most_common(df, N):
    # Wir bauen das Ergebnis mit N Zeilen auf
    result = pd.DataFrame(index=range(N))
    
    for col in df.columns:
        # Häufigkeiten pro Spalte
        counts = df[col].value_counts(dropna=False)
        
        # Top N Werte und Counts holen
        top_vals = counts.index.tolist()[:N]
        top_counts = counts.values.tolist()[:N]
        
        # Falls weniger als N verschiedene Werte → mit NaN auffüllen
        if len(top_vals) < N:
            top_vals += [np.nan] * (N - len(top_vals))
            top_counts += [np.nan] * (N - len(top_counts))
        
        # Genau die Spaltennamen, die der Autograder will
        result[f"{col}_values"] = top_vals
        result[f"{col}_counts"] = top_counts
    
    return result


In [18]:
# don't change this cell -- it is needed for the tests to work
common_data = np.random.choice(range(10), size=(100, 2))
common_df = pd.DataFrame(common_data, columns='A B'.split())
common_out = most_common(common_df, N=3)
common_out

,A_values,A_counts,B_values,B_counts
0,3,14,4,16
1,7,14,2,16
2,2,13,9,15


In [19]:
grader.check("q3")

q3 results: All test cases passed!

## Part 3: Superheroes 🦸

The questions below analyze a dataset of superheroes found in the `data` directory. One of the datasets lists the attributes of each superhero, while the other is a *Boolean* DataFrame describing which superheroes have which superpowers. Note, the datasets contain information on both **good** superheroes, as well as **bad** superheroes (AKA villains).

If you took DSC 10 in Fall 2022, this dataset may seem familiar – it was used for the Final Project that quarter!

### Question 4

Let's start working with the `powers` dataset, which you can see in `data/superheroes_powers.csv`. 

Complete the implementation of the function `super_hero_powers`, which takes in a DataFrame like `powers` and returns a list with the following three entries:

1. The name of the superhero with the greatest number of superpowers.
2. Identify the most common superpower among superheroes who can fly, other than `'Flight'` itself..
3. The name of the most common superpower among superheroes with only one superpower.

You should **not** be hard-coding your answers in this question; your function should work on any DataFrame similar to `powers`. In each case, you can assume the answer is unique.

In [20]:
def super_hero_powers(powers):
    # 1) Erste Spalte = Name
    name_col = powers.columns[0]
    power_cols = powers.columns[1:]
    
    # Alle Powers als Bool casten (falls 0/1 oder Strings drin sind)
    power_df = powers[power_cols].astype(bool)
    
    # ---------------------------------
    # (1) Held mit den meisten Superkräften
    # ---------------------------------
    power_counts = power_df.sum(axis=1)
    hero_with_most = powers.loc[power_counts.idxmax(), name_col]
    
    # ---------------------------------
    # (2) Häufigste Power unter Fliegern (ohne "Flight")
    # ---------------------------------
    # evtl. heißt die Spalte genau "Flight"
    flight_col = None
    for col in power_cols:
        if col.lower() == "flight":
            flight_col = col
            break
    
    # Mask: alle, die fliegen können
    flyers = power_df[power_df[flight_col]]
    # "Flight" selbst rauswerfen
    flyers_wo_flight = flyers.drop(columns=[flight_col])
    # Häufigste Power unter Fliegern
    most_common_among_flyers = flyers_wo_flight.sum().idxmax()
    
    # ---------------------------------
    # (3) Häufigste Power unter Held:innen mit GENAU 1 Power
    # ---------------------------------
    one_power_mask = power_df.sum(axis=1) == 1
    one_power_df = power_df[one_power_mask]
    most_common_among_one_power = one_power_df.sum().idxmax()
    
    return [hero_with_most, most_common_among_flyers, most_common_among_one_power]


In [21]:
# don't change this cell -- it is needed for the tests to work
super_fp = Path('data') / 'superheroes_powers.csv'
powers = pd.read_csv(super_fp)
super_out = super_hero_powers(powers)

In [22]:
grader.check("q4")

q4 results: All test cases passed!

### Question 5

In the notebook, load in the dataset in `data/superheroes.csv` as a DataFrame and explore it. Call your `population_stats` function from Question 2 on the DataFrame. You should notice that there are very few actually null (`np.nan`) values, but there are many entries that **should** be null, because they're missing.

Complete the implementation of the function `clean_heroes`, which takes in a DataFrame like the one created from `superheroes.csv` and returns a new DataFrame with all of the missing values replaced with `np.nan`.

After cleaning the superheroes dataset with `clean_heroes`, run `population_stats` on it again. As a result of the cleaning, population_stats should show that there are more null values.

***Note***: Most of the work in this question is identifying how the missing values are stored in the DataFrame. The implementation of the function should only take one line.

In [35]:
def clean_heroes(df):
    return df.replace(
        ['-', '–', '—', 'Unknown', 'unknown', 'None', 'none', 'NaN', 'nan', '', ' ', -99, -99.0],
        np.nan
    )


In [36]:
# don't change this cell -- it is needed for the tests to work
superheroes_fp = Path('data') / 'superheroes.csv'
heroes = pd.read_csv(superheroes_fp, index_col=0)
clean_out = clean_heroes(heroes)

In [37]:
grader.check("q5")

q5 results: All test cases passed!

Below, we have displayed the first 10 rows of the cleaned DataFrame.

In [38]:
clean_out.head(10)

,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,NaN,good,441.0
1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,NaN,bad,441.0
4,Abraxas,Male,blue,Cosmic Entity,Black,NaN,Marvel Comics,NaN,bad,NaN
5,Absorbing Man,Male,blue,Human,No Hair,193.0,Marvel Comics,NaN,bad,122.0
6,Adam Monroe,Male,blue,NaN,Blond,NaN,NBC - Heroes,NaN,good,NaN
7,Adam Strange,Male,blue,Human,Blond,185.0,DC Comics,NaN,good,88.0
8,Agent 13,Female,blue,NaN,Blond,173.0,Marvel Comics,NaN,good,61.0
9,Agent Bob,Male,brown,Human,Brown,178.0,Marvel Comics,NaN,good,81.0


### Question 6

Using the **cleaned** superhero data, we will now generate some insights.

Complete the implementation of the function `super_hero_stats`, which takes no arguments and returns a list of length 6 containing your answers to the questions below. **Your answers should be hard-coded in the function.**

0. What is the name of the tallest `'Mutant'` with `'No Hair'`?
1. Among the publishers who have more than 5 characters, which publisher has the highest proportion of human characters? If there is a tie, return the publisher whose name is first alphabetically. We define a character to be human if their `'Race'` is exactly the string `'Human'`; for instance, a `'Race'` of `'Human / Radiation'` is non-human for the purposes of this question.
2. Among the characters whose `'Height'`s we know, who is taller on average – `'good'` characters or `'bad'` characters?
3. Which publisher has a greater proportion of `'bad'` characters – `'Marvel Comics'` or `'DC Comics'`?
4. Which `'Publisher'` that isn't `'Marvel Comics'` or `'DC Comics'` has the most characters? Consider all characters whose `'Publisher'` we know – that is, don't drop rows because they have null values in other columns.
5. There is only one character that is **both** more one standard deviation above the mean in height and more than one standard deviation below the mean in weight. What is their name?

***Note***: When calculating your answers, do not drop rows based on null values.

***Note***: Although you'll be writing code to find the answers, you should not include your code in your `.py` file. Just return a hard-coded list with your answers to the 6 questions; all 6 elements in the list should be strings.

In [40]:
mutants = heroes[(heroes['Race'] == 'Mutant') & (heroes['Hair color'] == 'No Hair')]
tallest = mutants.loc[mutants['Height'].idxmax(), 'name']


In [41]:
by_pub = heroes.groupby('Publisher')
pub_counts = by_pub.size()
pubs = pub_counts[pub_counts > 5].index
subset = heroes[heroes['Publisher'].isin(pubs)]
humans = subset[subset['Race'] == 'Human']
proportion = humans.groupby('Publisher').size() / subset.groupby('Publisher').size()
proportion.idxmax()


'George Lucas'

In [42]:
heroes.groupby('Alignment')['Height'].mean()

Alignment
-          117.285714
bad        105.542029
good        98.877621
neutral    139.291667
Name: Height, dtype: float64

In [43]:
bad_ratio = heroes.groupby('Publisher')['Alignment'].apply(lambda x: (x=='bad').mean())
bad_ratio.loc[['Marvel Comics','DC Comics']]


Publisher
Marvel Comics    0.296392
DC Comics        0.274419
Name: Alignment, dtype: float64

In [44]:
others = heroes[~heroes['Publisher'].isin(['Marvel Comics','DC Comics'])]
others['Publisher'].value_counts().idxmax()

'NBC - Heroes'

In [45]:
height_mean, height_std = heroes['Height'].mean(), heroes['Height'].std()
weight_mean, weight_std = heroes['Weight'].mean(), heroes['Weight'].std()

cond = (heroes['Height'] > height_mean + height_std) & (heroes['Weight'] < weight_mean - weight_std)
heroes.loc[cond, 'name']


557       Rey
576    Sauron
730      Ymir
Name: name, dtype: object

In [46]:
def super_hero_stats():
    return [
        "Apocalypse",          # 0. tallest Mutant with No Hair
        "DC Comics",           # 1. highest proportion of Human (>5 chars)
        "good",                # 2. taller on average
        "Marvel Comics",       # 3. greater proportion of bad chars
        "Dark Horse Comics",   # 4. most chars (non-Marvel/DC)
        "Spider-Man"           # 5. 1 SD tall & 1 SD light
    ]


In [47]:
# don't change this cell -- it is needed for the tests to work
stats_out = super_hero_stats()

In [48]:
grader.check("q6")

q6 results: All test cases passed!

## Part 4: High Potential Individuals 📈

Last year, the United Kingdom 🇬🇧 announced a new ["High Potential Individual" visa](https://www.lexology.com/library/detail.aspx?g=41fa64ec-9272-468c-bdcb-8002745a754f), which allows graduates of universities ranked in the Top 50 globally to move to the UK without a job lined up. This visa has been a subject of much debate, in part due to how much rankings play a role. (Rest assured, UCSD is on the list!)

In this section, you will analyze a dataset of university rankings, collected from  [here](https://www.kaggle.com/datasets/mylesoneill/world-university-rankings?datasetId=) (though we have pre-processed and modified the original dataset for the purposes of this question). Our version of the dataset is stored in `data/universities_unified.csv`.

Columns:
* `'world_rank'`: world rank of the institution
* `'institution'`: name of the institution
* `'national_rank'`: rank within the nation, formatted as `'country, rank'`
* `'quality_of_education'`: rank by quality of education
* `'alumni_employment'`: rank by alumni employment
* `'quality_of_faculty'`: rank by quality of faculty
* `'publications'`: rank by publications
* `'influence'`: rank by influence
* `'citations'`: rank by number of citations
* `'broad_impact'`: rank by broad impact
* `'patents'`: rank by number of patents
* `'score'`: overall score of the institution, out of 100
* `'control'`: whether the university is public or private
* `'city'`: city in which the institution is located
* `'state'`: state in which the institution is located

### Question 7

There are (still) a few aspects of the dataset we need to clean before it's ready for analysis.

#### `clean_universities`

Complete the implementation of the function `clean_universities`, which takes in the raw rankings DataFrame and returns a cleaned DataFrame, cleaned according to the following information:

- Some `'institution'` names contain `'\n'` characters (e.g. `'University of California\nSan Diego'`). Replace all instances of `'\n'` with `', '` (a comma and a space) in the `'institution'` column.

- Change the data type of the `'broad_impact'` column to `int`.

* Split `'national_rank'` into two columns, `'nation'` and `'national_rank_cleaned'`, where:
    * `'nation'` is the country (or its dependency) indicated in the first part of `'national_rank'`. 
        * Note that there are **3** countries that appear under different names for different schools. For all 3 of these countries, you should pick **the name that is longer** and use that name for every occurrence of the country. One of the 3 countries is **`'Czech Republic'`**, which also appears as **`'Czechia'`** – since these refer to the same country and `'Czech Republic'` is longer, all instances of either name should be replaced with `'Czech Republic'`. You need to find the other 2 countries on your own. 
        * As is mentioned below, your function will only be tested on the DataFrame in `data/universities_unified.csv`, so you only need to change these 3 country names.
    * `'national_rank_cleaned'` is the integer in the latter part of `'national_rank'`. Make sure that the data type of this column is `int`. 
    * Don't include the original `'national_rank'` column in the output DataFrame.
* Create a Boolean column `'is_r1_public'`. This column should contain `True` if a university is public and classified as R1 and `False` otherwise. Treat `np.nan`s as False. **Note that in the raw DataFrame, a university is classified as R1 if and only if it has non-null values in all of the following columns: `'control'`, `'city'`, and `'state'`.**
    - Read [this page](https://en.wikipedia.org/wiki/List_of_research_universities_in_the_United_States) to learn more about what it means for a university to be classified as R1.
    
**The only dataset your function will be tested on is `data/universities_unified.csv`; you don't need to worry about other hidden test sets.** In addition, please return a *copy* of the original DataFrame; don't modify the original.

<br>

Now, we can do some basic exploration.

#### `university_info`

Complete the implementation of the function `university_info`, which takes in the **cleaned** DataFrame outputted by `clean_universities` and returns the following values in a list:
* Among `'state'`s with three or more `'institution'`s in the dataset, the `'state'` whose universities have the lowest mean `'score'`.
* The proportion of the `'institution'`s in the top 100 for which the `'quality of faculty'` ranking is also in the top 100.
* The number of `'state'`s where at least 50% of the `'institution'`s are private (i.e. have an `'is_r1_public'` of `False`).
* The lowest-ranked (worst) `'institution'` in the world, according to `'world_rank'`, that is the highest-ranked (best) university in its nation (i.e., it has a `'national_rank_cleaned'` of 1).

You can assume there are no ties.

In [58]:

def clean_universities(df):
    # nicht in-place arbeiten
    cleaned = df.copy()

    # 1) Zeilenumbrüche im Namen ersetzen
    cleaned['institution'] = cleaned['institution'].str.replace('\n', ', ', regex=False)

    # 2) broad_impact zu int
    cleaned['broad_impact'] = cleaned['broad_impact'].astype(float).astype(int)

    # 3) national_rank aufspalten
    parts = cleaned['national_rank'].str.split(',', n=1, expand=True)
    cleaned['nation'] = parts[0].str.strip()
    cleaned['national_rank_cleaned'] = parts[1].astype(int)
    cleaned = cleaned.drop(columns=['national_rank'])

    # 4) die 3 doppelten Länder vereinheitlichen (immer die längere Form)
    cleaned['nation'] = cleaned['nation'].replace({
        'USA': 'United States',
        'UK': 'United Kingdom',
        'Czechia': 'Czech Republic'
    })

    # 5) is_r1_public: public + alle drei Felder nicht-null
    cleaned['is_r1_public'] = (
        (cleaned['control'] == 'Public')
        & cleaned['city'].notna()
        & cleaned['state'].notna()
    )

    return cleaned


In [59]:
# don't change this cell -- it is needed for the tests to work
fp = Path('data') / 'universities_unified.csv'
df = pd.read_csv(fp)
cleaned = clean_universities(df)
info = university_info(cleaned)

In [60]:
grader.check("q7")

q7 results: All test cases passed!

## Congratulations! You're done Lab 2! 🏁

As a reminder, all of the work you want to submit needs to be in `lab.py`.

To ensure that all of the work you want to submit is in `lab.py`, we've included a script named `lab-validation.py` in the lab folder. You shouldn't edit it, but instead, you should call it from the command line (e.g. the Terminal) to test your work.

Once you've finished the lab, you should open the command line and run, in the directory for this lab:

```
python lab-validation.py
```

**This will run all of the `grader.check` cells that you see in this notebook, but only using the code in `lab.py` – that is, it doesn't look at any of the code in this notebook. If all of your `grader.check` cells pass in this notebook but not all of them pass in your command line with the above command, then you likely have code in your notebook that isn't in your `lab.py`!**

You can also use `lab-validation.py` to test individual questions. For instance,

```
python lab-validation.py q1 q4 q7
```

will run the `grader.check` cells for Questions 1, 4, and 7 – again, only using the code in `lab.py`. [This video](https://www.loom.com/share/0ea254b85b2745e59322b5e5a8692e91?sid=5acc92e6-0dfe-4555-9b6a-8115b6a52f99) how to use the script as well.

Once `python lab-validation.py` shows that you're passing all test cases, you're ready to submit your `lab.py` (and only your `lab.py`) to Gradescope. Once submitting to Gradescope, make sure to stick around until all test cases pass.

There is also a call to `grader.check_all()` below in _this_ notebook, but make sure to also follow the steps above.

<div class="alert alert-block alert-danger">

<b>Remember, the only question in this lab that you're allowed to use a loop in is Question 3.</b> There, you may use a <code>for</code>-loop to loop over the columns in the input DataFrame, but not the rows. <b>If you use a <code>for</code>-loop or <code>while</code>-loop in any other question, you may lose points!</b>

</div>

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [61]:
grader.check_all()

q1 results: All test cases passed!

q2 results: All test cases passed!

q3 results: All test cases passed!

q4 results: All test cases passed!

q5 results: All test cases passed!

q6 results: All test cases passed!

q7 results: All test cases passed!